This notebook generate plots of single trace examples of sharpening experiments.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import scipy.signal as signal
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib
%matplotlib notebook
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['axes.spines.top'] = False
matplotlib.rcParams['axes.spines.right'] = False
matplotlib.rcParams['legend.frameon'] = False

In [ ]:
dj.conn('at-database.ad.bcm.edu', 'shan')

In [ ]:
examples_main = [
    {'animal_id': 3019, 'slice_id': 2, 'cell_id': 7, 'trace_idx': 45},
    {'animal_id': 3019, 'slice_id': 2, 'cell_id': 7, 'trace_idx': 3},
    {'animal_id': 19049, 'slice_id': 1, 'cell_id': 1, 'trace_idx': 1},
    {'animal_id': 19049, 'slice_id': 1, 'cell_id': 1, 'trace_idx': 2},
    {'animal_id': 3019, 'slice_id': 3, 'cell_id': 2, 'trace_idx': 32},
    {'animal_id': 3019, 'slice_id': 3, 'cell_id': 2, 'trace_idx': 5},
    {'animal_id': 19049, 'slice_id': 1, 'cell_id': 8, 'trace_idx': 1},
    {'animal_id': 19049, 'slice_id': 1, 'cell_id': 8, 'trace_idx': 2},
    {'animal_id': 3109, 'slice_id': 2, 'cell_id': 4, 'trace_idx': 23},
    {'animal_id': 3109, 'slice_id': 2, 'cell_id': 4, 'trace_idx': 5},
    {'animal_id': 19049, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 1},
    {'animal_id': 19049, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 2},
]

In [ ]:
examples_L23_V1 = [
    {'animal_id': 2704, 'slice_id': 3, 'cell_id': 2, 'trace_idx': 23},
    {'animal_id': 2704, 'slice_id': 3, 'cell_id': 2, 'trace_idx': 29},
    {'animal_id': 2899, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 25},
    {'animal_id': 2899, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 12},
    {'animal_id': 3019, 'slice_id': 2, 'cell_id': 7, 'trace_idx': 35},
    {'animal_id': 3019, 'slice_id': 2, 'cell_id': 7, 'trace_idx': 2}
]

In [ ]:
def trace_clean(ax, example_vec, idx):
    query = example_vec[idx]
    time, trace, led, led_stat, ephys, baseline = (slicepatch.Firing & query).fetch1(
        'time', 'trace', 'led', 'led_stat', 'ephys', 'baseline')
    time = np.squeeze(time)
    trace = np.squeeze(trace)
    ephys = np.squeeze(ephys)
    trace_max = max(trace)
    trace_min = min(trace)
    unit = (trace_max - trace_min)/10
    
    if led_stat:
        led_on = time[np.where(led==1)[1]]
    else:
        led = (slicepatch.Firing & example_vec[idx-1]).fetch1('led')
        led_on = time[np.where(led==1)[1]]
    
    led_time_min = min(led_on)
    led_time_max = max(led_on)

    ephys_on = time[np.where(ephys==1)]
    ephys_time_min = min(ephys_on)
    ephys_time_max = max(ephys_on)

    current_base = baseline - 3*unit
    ephys_trace = current_base + ephys*unit
    
    if led_stat:
        color = 'lightblue'
    else:
        color = 'lightgray'
        
    ax.fill_betweenx(y=[trace_min, trace_max], x1=led_time_min, x2=led_time_max, color=color)
    ax.plot(time, trace, color='black', linewidth=1)
    ax.plot(time, ephys_trace, color='black', linewidth=1)
    ax.set_xlim([ephys_time_min-25, ephys_time_max +50])
    ax.axis('off')
    ax.text(ephys_time_min-50, current_base, 'I')
    
    if idx==0:
        ax.text(np.mean([ephys_time_min, ephys_time_max])-50, current_base-1.5*unit, '300 ms')
        ax.text(led_time_min-5, trace_max+0.5*unit, '20 ms')

In [ ]:
fsize_label = 12
pos = dict(left=0.15, top=0.8, bottom=0.08, right=0.9)
fig, axs = plt.subplots(3, 4, figsize=[7.5, 3.5])

for iax, ax in enumerate(axs.flatten()):
    trace_clean(ax, examples_main, iax)

fig.subplots_adjust(**pos)
fig.text(0.28, 0.94, r'LM to V1', fontsize=fsize_label)
fig.text(0.66, 0.94, r'vM1 to vS1', fontsize=fsize_label)
fig.text(0.01, 0.95, 'A', fontsize=14)

layer_label_left = 0.04
fig.text(layer_label_left, 0.73, r'L2/3 cell', fontsize=fsize_label)
fig.text(layer_label_left, 0.44, r'L4 cell', fontsize=fsize_label)
fig.text(layer_label_left, 0.15, r'L5 cell', fontsize=fsize_label)
fig.text(0.2, 0.85, 'LED on')
fig.text(0.4, 0.85, 'LED off')
fig.text(0.59, 0.85, 'LED on')
fig.text(0.79, 0.85, 'LED off')


# PSTH for individual cells

In [ ]:
examples_L23_V1 = [
    {'animal_id': 2704, 'slice_id': 3, 'cell_id': 2},
    {'animal_id': 2899, 'slice_id': 1, 'cell_id': 2},
    {'animal_id': 3138, 'slice_id': 1, 'cell_id': 7}
]

In [ ]:
def plot_psth_ind(ax, cells, idx):
    
    cell = cells[idx].copy()
    if 'trace_idx' in cell.keys(): 
        cell.pop('trace_idx')
    aligned_spk_times_on = (slicepatch.SpkTimes & cell & 
                            (slicepatch.Firing & 'led_stat=1')).fetch('aligned_spk_times')
    aligned_spk_times_off = (slicepatch.SpkTimes & cell & 
                             (slicepatch.Firing & 'led_stat=0')).fetch('aligned_spk_times')
    spk_times_on = np.squeeze(np.hstack(aligned_spk_times_on))
    n_on = len(aligned_spk_times_on)
    spk_times_off = np.squeeze(np.hstack(aligned_spk_times_off))
    n_off = len(aligned_spk_times_off)
    time_range = [-25, 81]
    dt = 2
    nbins = int((time_range[1]-time_range[0])/dt)
    time = list(range(time_range[0], time_range[1], dt))
    psth_on = np.divide(np.histogram(spk_times_on, range=time_range, bins=nbins)[0], n_on*dt*0.001)
    psth_off = np.divide(np.histogram(spk_times_off, range=time_range, bins=nbins)[0], n_off*dt*0.001)
    
    ax.plot(time, psth_on, color='r', linewidth=1, label='LED on')
    ax.plot(time, psth_off, color='black', linewidth=1, label='LED off')
    ylim = ax.get_ylim()
    ax.fill_between([0, 20], ylim[0], ylim[1], color='lightblue', label='LED range', zorder=1)
    
    if iax == 2:
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=7, frameon=False)
    
    if iax == 8:
        ax.set_xlabel('Time relative to light stim onset (ms)')
    
    if iax == 5:
        ax.set_ylabel('Firing rate (spks/s)')

In [ ]:
example = [{'animal_id': 19159, 'slice_id': 2, 'cell_id': 23}]
f, ax = plt.subplots(1, 1, figsize=[5, 3])
plot_psth_ind(ax, example, 0)

In [ ]:
fsize_label = 12
pos = dict(left=0.1, top=0.9, bottom=0.1, right=0.85)
example_groups_V1 = [
    [{'animal_id': 2704, 'slice_id': 3, 'cell_id': 2, 'trace_idx': 23},
     {'animal_id': 2704, 'slice_id': 3, 'cell_id': 2, 'trace_idx': 29},
     {'animal_id': 2899, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 25},
     {'animal_id': 2899, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 12},
     {'animal_id': 3138, 'slice_id': 1, 'cell_id': 7, 'trace_idx': 24},
     {'animal_id': 3138, 'slice_id': 1, 'cell_id': 7, 'trace_idx': 5}],
    [{'animal_id': 2899, 'slice_id': 2, 'cell_id': 3, 'trace_idx': 12},
     {'animal_id': 2899, 'slice_id': 2, 'cell_id': 3, 'trace_idx': 1},
     {'animal_id': 3138, 'slice_id': 2, 'cell_id': 3, 'trace_idx': 10},
     {'animal_id': 3138, 'slice_id': 2, 'cell_id': 3, 'trace_idx': 2},
     {'animal_id': 3288, 'slice_id': 2, 'cell_id': 3, 'trace_idx': 13},
     {'animal_id': 3288, 'slice_id': 2, 'cell_id': 3, 'trace_idx': 5}],
    [{'animal_id': 2704, 'slice_id': 3, 'cell_id': 6, 'trace_idx': 9},
     {'animal_id': 2704, 'slice_id': 3, 'cell_id': 6, 'trace_idx': 8},
     {'animal_id': 3019, 'slice_id': 3, 'cell_id': 6, 'trace_idx': 32},
     {'animal_id': 3019, 'slice_id': 3, 'cell_id': 6, 'trace_idx': 5},
     {'animal_id': 19467, 'slice_id': 1, 'cell_id': 6, 'trace_idx': 1},
     {'animal_id': 19467, 'slice_id': 1, 'cell_id': 6, 'trace_idx': 3}]
]
example_groups_S1 = [
    [{'animal_id': 19049, 'slice_id': 1, 'cell_id': 7, 'trace_idx': 5}, # L23
     {'animal_id': 19049, 'slice_id': 1, 'cell_id': 7, 'trace_idx': 4},
     {'animal_id': 19159, 'slice_id': 1, 'cell_id': 1, 'trace_idx': 5},
     {'animal_id': 19159, 'slice_id': 1, 'cell_id': 1, 'trace_idx': 4},
     {'animal_id': 19159, 'slice_id': 2, 'cell_id': 21, 'trace_idx': 3},
     {'animal_id': 19159, 'slice_id': 2, 'cell_id': 21, 'trace_idx': 2}],
    [{'animal_id': 19049, 'slice_id': 1, 'cell_id': 9, 'trace_idx': 5}, # L4
     {'animal_id': 19049, 'slice_id': 1, 'cell_id': 9, 'trace_idx': 2},
     {'animal_id': 19159, 'slice_id': 2, 'cell_id': 17, 'trace_idx': 1},
     {'animal_id': 19159, 'slice_id': 2, 'cell_id': 17, 'trace_idx': 2},
     {'animal_id': 19159, 'slice_id': 2, 'cell_id': 17, 'trace_idx': 1},
     {'animal_id': 19159, 'slice_id': 2, 'cell_id': 19, 'trace_idx': 1},
     {'animal_id': 19159, 'slice_id': 2, 'cell_id': 19, 'trace_idx': 2}], 
    [{'animal_id': 14789, 'slice_id': 1, 'cell_id': 7, 'trace_idx': 25}, # L5 # bursty
     {'animal_id': 14789, 'slice_id': 1, 'cell_id': 7, 'trace_idx': 12},
     {'animal_id': 14796, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 20},
     {'animal_id': 14796, 'slice_id': 1, 'cell_id': 2, 'trace_idx': 2},
     {'animal_id': 19049, 'slice_id': 1, 'cell_id': 6, 'trace_idx': 3},
     {'animal_id': 19049, 'slice_id': 1, 'cell_id': 6, 'trace_idx': 4}]
]

In [ ]:
labels = ['L2/3', 'L4', 'L5']
panels = ['A', 'B', 'C']

areas = ['V1', 'vS1']

for area in areas:
    if area == 'V1':
        example_groups = example_groups_V1
    else:
        example_groups = example_groups_S1

    for example_group, label, panel in zip(example_groups, labels, panels):
        fig, axs = plt.subplots(3, 3, figsize=[9, 5])
        for iax, ax in enumerate(axs.flatten()):
            if np.mod(iax+1, 3):
                trace_clean(ax, example_group, iax - int((iax+1)/3))
            else:
                plot_psth_ind(ax, example_group, iax - int((iax+1)/3))
    
        fig.tight_layout()
        fig.subplots_adjust(**pos)
        fig.text(0.04, 0.5, label + ' ' + area)
        fig.text(0.01, 0.95, panel, fontsize=14)
        fig.text(0.18, 0.93, 'LED on')
        fig.text(0.45, 0.93, 'LED off')
        
        if label=='L2/3':
            fig.savefig('L23_' + area + '_examples_supp.png', dpi=300)
        else:
            fig.savefig(label + '_' + area+ '_examples_supp.png', dpi=300)

In [ ]:
fig.subplots_adjust(**pos)

In [ ]:
example_group

In [ ]:
trace = [{'animal_id': 19049, 'slice_id': 1, 'cell_id': 9, 'trace_idx': 5}]

f, ax = plt.subplots(1,1, figsize=[5,3])
trace_clean(ax, trace, 0)

In [ ]:
slicepatch.Firing & {'animal_id': 19049, 'slice_id': 1, 'cell_id': 9, 'trace_idx':2}